## MH4518 Group Project

Product: https://derivative.credit-suisse.com/ch/ch/en/detail/drop-back-certificate-s-p-500/CH1199361879/119936187


### 1. Initial Set-up

<ol>
<li> Set directories
<li> Install dependencies
<li> Import libraries
<li> Suppress Warnings (Optional)
</ol>

NOTE: Ensure you are using the correct kernel to run the Jupyter Notebook with the correctly set root driectory.

In [1]:
import os, sys

# Set the root directory accordingly
ROOT_DIR = "D:\Coding\MH4518"
os.chdir(ROOT_DIR)
#sys.path.append(ROOT_DIR)

# These directories are set relative to root
DATA_DIR = os.path.join(ROOT_DIR, "data")
SAVE_DIR = os.path.join(ROOT_DIR, "simulations")

In [2]:
# Install additional libraries using pip
#! pip install -r requirements.txt

# Or if you have Conda installed (reccommended)
# conda env create -f environment.yml

In [3]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_market_calendars as mcal

from numba import jit, njit
from nelson_siegel_svensson.calibrate import calibrate_ns_ols
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [4]:
# OPTIONAL: Suppresing the deprecation notices while using numba and matplotlib (reccommended)
import warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

warnings.filterwarnings('ignore', module = "matplotlib\..*" )
warnings.filterwarnings('ignore', category=NumbaDeprecationWarning)
warnings.filterwarnings('ignore', category=NumbaPendingDeprecationWarning)

### 2. Loading Data

The asset and product prices as well as the interest and dividend rates were collected and preprocessed using the notebook present in the `/data` directory.

<ol>
<li> daily_interest_rates.csv
<li> asset_prices_processed.csv
<li> product_prices_processed.csv
<li> SPY_quarterly_dividend_rates.csv
<li> VIX_annual_implied_volatility_rates.csv
</ol>
<br>

However for our simulations we will only focus on the following time period,

<li> Start date: 09-08-2022
<li> End date: 09-11-2023

In [5]:
# Load asset prices
asset_prices = pd.read_csv(os.path.join(DATA_DIR, "asset_prices_processed.csv"))
asset_prices["date"] = pd.to_datetime(asset_prices["date"])
asset_prices = asset_prices.set_index("date")

# Load product prices
product_prices = pd.read_csv(os.path.join(DATA_DIR, "product_prices_processed.csv"))
product_prices["date"] = pd.to_datetime(product_prices["date"])
product_prices = product_prices.set_index("date")

# Load daily interest rates
interest_rates = pd.read_csv(os.path.join(DATA_DIR, "daily_interest_rates.csv"))
interest_rates["date"] = pd.to_datetime(interest_rates["date"])
interest_rates = interest_rates.set_index("date")

# Load quarterly dividend rates
dividend_rates = pd.read_csv(os.path.join(DATA_DIR, "SPY_quarterly_dividend_rates.csv"))
dividend_rates["date"] = pd.to_datetime(dividend_rates["date"])
dividend_rates = dividend_rates.set_index("date")

# Load annual implied volatility rates
implied_volatility_rates = pd.read_csv(os.path.join(DATA_DIR, "VIX_annual_implied_volatility_rates.csv"))
implied_volatility_rates["date"] = pd.to_datetime(implied_volatility_rates["date"])
implied_volatility_rates = implied_volatility_rates.set_index("date")

In [6]:
# Visualizing the implied volatility rates (for example)
implied_volatility_rates.head()

,value
date,
2022-07-19,24.500000
2022-07-20,23.879999
2022-07-21,23.110001
2022-07-22,23.030001
2022-07-25,23.360001


### 3. Payoff Function

In [7]:
def find_trigger_price(S_path, trigger):
    '''
    Returns the price that triggered the event, if it exists, else None.
    '''
    # Return the first price that triggers the event
    for price in S_path:
        if price <= trigger:
            return price
        
    return None


def calculate_payoff(S_path, Y_0 = 1000, rate = 0.0985, years = 3):
    '''
    Computes the payoff of a simulated path.
    '''
    S_0 = 3790.38   # From factsheet
    S_t1 = find_trigger_price(S_path, trigger = 3411.3420)   # 0.90 * S_0
    S_t2 = find_trigger_price(S_path, trigger = 3221.8230)   # 0.85 * S_0
    S_t3 = find_trigger_price(S_path, trigger = 3032.3040)   # 0.80 * S_0
    S_T = S_path[-1]

    # Edge Case
    if(S_t3 == 0 or S_t2 == 0 or S_t1 == 0):
        return 0

    # 1. Fixed returns independent of trigger events
    Y_T = (0.55*Y_0) * (S_T/S_0)

    # 2. Adjusted returns after trigger events
    if S_t3 is not None:
        multiplier = (S_T/S_t1 + S_T/S_t2 + S_T/S_t3)
    elif S_t2 is not None:
        multiplier = (S_T/S_t1 + S_T/S_t2)
    elif S_t1 is not None:
        multiplier = (S_T/S_t1)
    else:
        multiplier = 0.0
    Y_T += (0.15*Y_0) * multiplier

    # 3. Daily accrued interest of 9.85% p.a.
    if S_t3 is not None:
        principal = 0.0
    elif S_t2 is not None:
        principal = (0.15*Y_0)
    elif S_t1 is not None:
        principal = (0.30*Y_0)
    else:
        principal = (0.45*Y_0)
    Y_T += principal * (1 + rate * years)
    
    assert Y_T >= 0.0   # Sanity Check
    return Y_T

In [8]:
# OPTIONAL: Testing the payoff function with values in the factsheet (Flawed, all triggers are hit)
S_0 = 3790.38
S_path_partial = [3790.41, 3800.15, 3373.44, 4000.00, 3183.92, 2994.40]
S_T = [0.0, 2653.27, 3032.3, 3411.34, 3790.38, 4169.42, 4548.46, 4927.49]

for final_closing_price in S_T:
    S_path = S_path_partial + [final_closing_price]
    payoff = calculate_payoff(S_path)

    print(f"S_T = {final_closing_price} \t   Performance of Asset = {100*(final_closing_price - S_0)/S_0:.2f}% \t Performance of Certificate = {(payoff - 1000)/10:.2f}% \t Payoff = {payoff:.2f}")

S_T = 0.0 	   Performance of Asset = -100.00% 	 Performance of Certificate = -100.00% 	 Payoff = 0.00
S_T = 2653.27 	   Performance of Asset = -30.00% 	 Performance of Certificate = -23.91% 	 Payoff = 760.89
S_T = 3032.3 	   Performance of Asset = -20.00% 	 Performance of Certificate = -13.04% 	 Payoff = 869.59
S_T = 3411.34 	   Performance of Asset = -10.00% 	 Performance of Certificate = -2.17% 	 Payoff = 978.29
S_T = 3790.38 	   Performance of Asset = 0.00% 	 Performance of Certificate = 8.70% 	 Payoff = 1086.98
S_T = 4169.42 	   Performance of Asset = 10.00% 	 Performance of Certificate = 19.57% 	 Payoff = 1195.68
S_T = 4548.46 	   Performance of Asset = 20.00% 	 Performance of Certificate = 30.44% 	 Payoff = 1304.38
S_T = 4927.49 	   Performance of Asset = 30.00% 	 Performance of Certificate = 41.31% 	 Payoff = 1413.08


### 4. Geometric Brownian Motion

Implmentation of the Black-Scholes model for pricing. THe form derived from Ito's lemma was used.

In [9]:
@njit
def simulate_GBM(num_sim, S_0, r, sigma, delta_t, T, Z):  # NOTE: For exact GMB simply set r = mu while calling
  """
  r is the daily interest rate
  sigma is daily volatility
  T is actually tau = T - t (in years)
  delta_t is 1 / 252
  Z is the random variates sampled from N(0, 1)
  """
  # Initializations
  v = r - 0.5*(sigma**2)
  num_periods = int(T/delta_t)
  S_matrix = np.zeros(shape=(num_sim, num_periods + 1), dtype=np.float64)

  # We are using the form derived by Ito's lemma 
  for i in range(num_sim):
    S_matrix[i][0] = S_0
    for j in range(1, num_periods + 1):
      log_diff = v*delta_t + (sigma*np.sqrt(delta_t) * Z[i][j-1])
      S_matrix[i][j] = S_matrix[i][j-1]*np.exp(log_diff)

  return S_matrix

### 5. Variance Reduction for GBM

Functions that implement,

<ol> 
<li> Antithetic Variates
<li> Control Variates
<li> Empirical Martingale Simulation
</ol>

In [10]:
@njit
def simulate_GBM_AV(num_sim, S_0, r, sigma, delta_t, T, Z):
  """
  r is the daily interest rate
  sigma is daily volatility
  T is actually tau = T - t (in years)
  delta_t is 1 / 252
  Z is the random variates sampled from N(0, 1)
  """
  # Initializations
  v = r - 0.5*(sigma**2)
  num_periods = int(T/delta_t)
  S_matrix = np.full(shape=(num_sim, num_periods + 1), fill_value = S_0, dtype=np.float64)
  S_tilde_matrix = np.full(shape=(num_sim, num_periods + 1), fill_value = S_0, dtype=np.float64)

  # Ito's lemma form (just a bit more concise)
  for i in range(num_sim):
    S_matrix[i][0] = S_0
    S_tilde_matrix[i][0] = S_0
    for j in range(1, num_periods + 1):
        S_matrix[i][j] = S_matrix[i][j-1]*np.exp(v*delta_t + (sigma*np.sqrt(delta_t) * Z[i][j-1]))
        S_tilde_matrix[i][j] = S_tilde_matrix[i][j-1]*np.exp(v*delta_t - (sigma*np.sqrt(delta_t) * Z[i][j-1]))  # Negative Z(.)  
  
  return np.vstack((S_matrix, S_tilde_matrix))

In [11]:
@njit
def simulate_GBM_EMS(num_sim, S_0, r, sigma, delta_t, T, Z):
  """
  r is the daily interest rate
  sigma is daily volatility
  T is actually tau = T - t (in years)
  delta_t is 1 / 252
  Z is the random variates sampled from N(0, 1)
  """
  # Initializations
  num_periods = int(T/delta_t)
  S_matrix = np.full(shape=(num_sim, num_periods + 1), fill_value = S_0, dtype=np.float64)
  Z_matrix = np.full(shape=(num_sim + 1, num_periods + 1), fill_value = S_0, dtype=np.float64)

  # Simulating the GBM path and set the first coloumn of S_matrix
  gbm_path = simulate_GBM(num_sim, S_0, r, sigma, delta_t, T, Z=Z)
  S_matrix[:, 0] = gbm_path[:, 0]
  
  # Correcting the path using EMS
  for j in range(1, num_periods+1):
    Z_matrix[:num_sim, j-1] = (S_matrix[:, j-1] * gbm_path[:, j]) / gbm_path[:, j-1]
    Z_matrix[num_sim, j-1] = np.exp(-r * ((j-1) * delta_t)) * np.mean(Z_matrix[:num_sim, j-1])
    S_matrix[:, j] = (gbm_path[:, 0] * Z_matrix[:num_sim, j-1]) / Z_matrix[num_sim, j-1]

  return S_matrix

In [12]:
@jit  # Need to use some pythonic capabilities for this function
def simulate_GBM_CV(num_sim, S_0, r, sigma, delta_t, T, Z_pilot, Z_new, use_minimum=False):
  """
  r is the daily interest rate
  ...
  N_1 == N_2 == num_sim in our implementation
  If use_minmum = True, we set S_T as the respective minimum price of for each GBM path
  """
  # Disambiguation of r and sigma
  r_daily, r_yearly = r, (r * 252)     # Basically r_yearly is used in any exp(.) and r_daily is for the gbm simulations.
  sigma_daily, sigma_yearly = sigma, (252 ** (1/2)) * sigma

  # Generating a pilot of N_1 prices
  gbm_path_pilot = simulate_GBM(num_sim, S_0, r_daily, sigma_daily, delta_t, T, Z=Z_pilot)
  payoff_pilot = np.exp(-r_yearly*T)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=gbm_path_pilot)

  # Calulating c*
  S_T_vector_pilot = [min(path) for path in gbm_path_pilot] if use_minimum else [path[-1] for path in gbm_path_pilot]
  covariance_matrix = np.cov(payoff_pilot, S_T_vector_pilot)
  c_star = -covariance_matrix[0][1]/(S_0**2 * np.exp(2*r_yearly*T) * (np.exp(sigma_yearly**2 * T) - 1))    # By the martingale property
  
  # Simulating N_2 new prices
  gbm_path_new = simulate_GBM(num_sim, S_0, r_daily, sigma_daily, delta_t, T, Z=Z_new)
  payoff_new = np.exp(-r_yearly*T)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=gbm_path_new)
  
  # Finally, computing the control variates
  S_T_vector_new = [min(path) for path in gbm_path_new] if use_minimum else [path[-1] for path in gbm_path_new]
  payoff_CV = payoff_new + c_star * (S_T_vector_new - S_0 * np.exp(r_yearly*T))

  return payoff_CV

### 6. Backtesting GBM

We take 252 day long windows of data for each iteration.

eg: The first window begins on 9th August, 2022 and ends on 9th August, 2023 and we price the product on 10th August.

#### 6.1 Parameters and Rolling Window

In [13]:
# Create a market calendar for date handling
nyse = mcal.get_calendar('NYSE')
product_lifetime = nyse.schedule(start_date='2022-07-14', end_date='2025-07-14')
print(len(product_lifetime))

753


In [14]:
# Parameters for each set of simulations
num_sim = 20000    # Push to 1,000,000
delta_t = 1/252
num_periods = len(product_lifetime)
T = num_periods/252
# T is in years
# delta t_is also in years

In [15]:
# One set of simulations for each date in [start_date, end_date]
window_size = 252
start_date = pd.to_datetime("2023-08-09")
end_date = pd.to_datetime("2023-11-09")

simulation_dates = product_prices.index[(product_prices.index >= start_date) & (product_prices.index <= end_date)]
print(f"{len(simulation_dates)} days")
print(f"Start Date: {simulation_dates[0]}; End date: {simulation_dates[-1]}")

67 days
Start Date: 2023-08-09 00:00:00; End date: 2023-11-09 00:00:00


In [16]:
# We shall sample a window of size 252 from the historical data (EXCLUDING the current date)
backtest_windows = []   # Prices only
backtest_windows_with_dates = []
previous_window_end_date = None

for window_end_date in simulation_dates:
    # For each date the product was prices, we need 252 points BEFORE this date
    try:
        window_start_index = asset_prices.index.get_loc(window_end_date) - window_size
    except:
        window_start_index = asset_prices.index.get_loc(previous_window_end_date) - window_size + 1
        print(f"Asset not priced on {window_end_date}. Using data up till the previous date!")

    assert window_start_index >= 0

    # Slice the dataset according to the window
    window_start_date = asset_prices.index[window_start_index]
    window_asset_data = asset_prices[(asset_prices.index >= window_start_date) & (asset_prices.index < window_end_date)]

    # Collect all the windows
    assert len(window_asset_data) == window_size
    backtest_windows_with_dates.append(window_asset_data)   # For validatiuon purposes
    backtest_windows.append(np.array(window_asset_data["value"]))
    
    # For handling edge cases in the try-except block
    previous_window_end_date = window_end_date

print(f"Collected data for {len(backtest_windows)} windows of size {window_size} each.")

Asset not priced on 2023-09-04 00:00:00. Using data up till the previous date!
Asset not priced on 2023-11-09 00:00:00. Using data up till the previous date!
Collected data for 67 windows of size 252 each.


In [17]:
# Validating the rolling window boundaries
for x, y in zip(simulation_dates, backtest_windows_with_dates):
    print(f"Pricing asset on - \t\t{x}.")
    print(f"We have data till - \t{y.index[-1]}.\n")

Pricing asset on - 		2023-08-09 00:00:00.
We have data till - 	2023-08-08 00:00:00.

Pricing asset on - 		2023-08-10 00:00:00.
We have data till - 	2023-08-09 00:00:00.

Pricing asset on - 		2023-08-11 00:00:00.
We have data till - 	2023-08-10 00:00:00.

Pricing asset on - 		2023-08-14 00:00:00.
We have data till - 	2023-08-11 00:00:00.

Pricing asset on - 		2023-08-15 00:00:00.
We have data till - 	2023-08-14 00:00:00.

Pricing asset on - 		2023-08-16 00:00:00.
We have data till - 	2023-08-15 00:00:00.

Pricing asset on - 		2023-08-17 00:00:00.
We have data till - 	2023-08-16 00:00:00.

Pricing asset on - 		2023-08-18 00:00:00.
We have data till - 	2023-08-17 00:00:00.

Pricing asset on - 		2023-08-21 00:00:00.
We have data till - 	2023-08-18 00:00:00.

Pricing asset on - 		2023-08-22 00:00:00.
We have data till - 	2023-08-21 00:00:00.

Pricing asset on - 		2023-08-23 00:00:00.
We have data till - 	2023-08-22 00:00:00.

Pricing asset on - 		2023-08-24 00:00:00.
We have data till - 	20

#### 6.2 Interpolating the Interest Rate Curves

In [18]:
maturities = np.array([1/12, 2/12, 3/12, 4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
interest_rates.columns = maturities

# The 4 month interest rate is a bit sparse
interest_rates = interest_rates.dropna(how='any', axis=1)
maturities = interest_rates.columns.to_numpy()
interest_rates.shape

(330, 12)

In [19]:
curves = []
for date in simulation_dates:
    try:
        interest_rate = interest_rates.loc[date].to_numpy()
        curve_fit, status = calibrate_ns_ols(maturities, interest_rate)
        curves.append(curve_fit)

    except KeyError:
        # Handle the case where the rate is not found
        print(f"Rate not found for date {date}, using the rate from the previous date.")
        previous_date = interest_rates.index[interest_rates.index <= date].max()
        corrected_rate = interest_rates.loc[previous_date].to_numpy()
        curve_fit, status = calibrate_ns_ols(maturities, corrected_rate)
        curves.append(curve_fit)

    except Exception as e:
        print(f"An error occurred for date {date}: {e}")

Rate not found for date 2023-09-04 00:00:00, using the rate from the previous date.
Rate not found for date 2023-10-09 00:00:00, using the rate from the previous date.


#### 6.3 Helper Functions

In [20]:
# Used for computing Historical Volatility
def get_lognormal_statistics(prices, delta_t):
    log_data = np.log(prices)
    log_returns = log_data[1:] - log_data[:-1]
    mu = np.mean(log_returns) / delta_t
    sigma = np.std(log_returns, ddof=1) / np.sqrt(delta_t)
    return mu, sigma

In [21]:
# Used for querying the annual Implied Volatility rate
def get_implied_volatility_rate(date):
    # We only collected the rates relavant for our timeframe
    assert date > pd.to_datetime('2022-07-19') and date <= pd.to_datetime('2023-11-09')

    try:
        # Get the rate for the previous date (if we're predicting for tomorrow, we only have data up till yesterday)
        previous_entry = implied_volatility_rates.loc[:date].iloc[-1]
        implied_volatility = previous_entry['value']

    except Exception as e:
        print(f"An error occurred for date {date}: {e}")

    return implied_volatility

In [22]:
def get_dividend_rate(date):
    # We only collected the rates relavant for our timeframe
    assert date > pd.to_datetime('2022-06-17')

    # Find the relavant quarter and return the dividend rate
    for quarter_date in dividend_rates.index:
        if date >= quarter_date:
            return dividend_rates.loc[quarter_date, 'dividends']

    print("Returning last rate!")
    return dividend_rates.loc[quarter_date, 'dividend_rate']

In [24]:
# Generating a common random variable/common randomness matrix
Z_matrix = np.random.normal(0, 1, size=(len(simulation_dates), num_sim, num_periods))

MemoryError: Unable to allocate 7.52 GiB for an array with shape (67, 20000, 753) and data type float64

#### 6.4 Running the Simulations (aka The Exciting Part)

#### 6.4.1 GBM with Dividends Adjustment

This will be our baseline model.

In [ ]:
# GBM with Dividends without Variance Reduction
GBM_Div_expected_values = []

for date, prices in zip(simulation_dates, backtest_windows):
  # Convert date to index (so we can lookup prices and interest rate curves)
  date_index = simulation_dates.get_loc(date)

  # Estimate r
  delta = len(product_lifetime[date:])
  tau = delta / 252    # time diff in years
  curve_fit = curves[date_index]
  r_yearly = curve_fit(tau) / 100    # yearly (interpolated) interest rate

  # Calulcate mu and sigma (we discard mu)
  _, sigma_daily = get_lognormal_statistics(prices, delta_t)    # sigma is daily variance

  # IMPORTANT: Adjust the interest rate based on dividends
  r_adjusted_yearly = r_yearly - get_dividend_rate(date) / 100
  r_adjusted_daily = r_adjusted_yearly / 252

  # Simulate prices, calculate payoff and expected value
  simulated_prices = simulate_GBM(num_sim, prices[-1], r_adjusted_daily, sigma_daily, delta_t, tau, Z=Z_matrix[date_index])
  payoffs = np.exp(-r_adjusted_yearly*tau)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices)
  expected_value = np.mean(payoffs)
  
  GBM_Div_expected_values.append(expected_value)

#### 6.4.2 GBM with Implied Volatility

In [ ]:
# GBM with Implied Volatility without Variance Reduction
GBM_Vol_expected_values = []

for date, prices in zip(simulation_dates, backtest_windows):
  # Convert date to index (so we can lookup prices and interest rate curves)
  date_index = simulation_dates.get_loc(date)

  # Estimate r
  delta = len(product_lifetime[date:])
  tau = delta / 252    # time diff in years
  curve_fit = curves[date_index]
  r_yearly = curve_fit(tau) / 100    # yearly (interpolated) interest rate

  # Set sigma to the daily implied volatility rate
  sigma_yearly = get_implied_volatility_rate(date) / 100
  sigma_daily = sigma_yearly / (252**(1/2))

  # We need the daily rate for the simulations
  r_daily = r_yearly / 252

  # Simulate prices, calculate payoff and expected value
  simulated_prices = simulate_GBM(num_sim, prices[-1], r_daily, sigma_daily, delta_t, tau, Z=Z_matrix[date_index])
  payoffs = np.exp(-r_yearly*tau)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices)
  expected_value = np.mean(payoffs)

  GBM_Vol_expected_values.append(expected_value)

#### 6.4.3 Variance Reduction Techniques

We tried, 

<ol> 
<li> Antithetic Variates
<li> Control Variates
<li> Empirical Martingale Simulation
</ol>

<br>
NOTE: All of these use were applied on the Implied Volatility Model. 

In [ ]:
# GBM with Implied Volatility and Dividends and Antithetic Variance Reduction
GBM_Vol_AV_expected_values = []

for date, prices in zip(simulation_dates, backtest_windows):
  # Convert date to index (so we can lookup prices and interest rate curves)
  date_index = simulation_dates.get_loc(date)

  # Estimate r
  delta = len(product_lifetime[date:])
  tau = delta / 252
  curve_fit = curves[date_index]
  r_yearly = curve_fit(tau) / 100

  # Set sigma to the daily implied volatility rate
  sigma_yearly = get_implied_volatility_rate(date) / 100
  sigma_daily = sigma_yearly / (252**(1/2))

  # We need the daily rate for the simulations
  r_daily = r_yearly / 252
  
  # Simulate prices, calculate payoff and expected value
  simulated_prices = simulate_GBM_AV(int(num_sim/2), prices[-1], r_daily, sigma_daily, delta_t, tau, Z=Z_matrix[date_index])
  payoffs = np.exp(-r_yearly*tau)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices)
  expected_value = np.mean(payoffs)
  
  GBM_Vol_AV_expected_values.append(expected_value)

In [ ]:
# GBM with Implied Volatility and Dividends and EMS Variance Reduction
GBM_Vol_EMS_expected_values = []

for date, prices in zip(simulation_dates, backtest_windows):
  # Convert date to index (so we can lookup prices and interest rate curves)
  date_index = simulation_dates.get_loc(date)

  # Estimate r
  delta = len(product_lifetime[date:])
  tau = delta / 252
  curve_fit = curves[date_index]
  r_yearly = curve_fit(tau) / 100

  # Set sigma to the daily implied volatility rate
  sigma_yearly = get_implied_volatility_rate(date) / 100
  sigma_daily = sigma_yearly / (252**(1/2))

  # We need the daily rate for the simulations
  r_daily = r_yearly / 252

  # Simulate prices, calculate payoff and expected value
  simulated_prices = simulate_GBM_EMS(num_sim, prices[-1], r_daily, sigma_daily, delta_t, tau, Z=Z_matrix[date_index])
  payoffs = np.exp(-r_yearly*tau)*np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices)
  expected_value = np.mean(payoffs)
  
  GBM_Vol_EMS_expected_values.append(expected_value)

In [ ]:
# GBM with Implied Volatility and Dividends and Control Variate Variance Reduction
GBM_Vol_CV_expected_values = []

for date, prices in zip(simulation_dates, backtest_windows):
  # Convert date to index (so we can lookup prices and interest rate curves)
  date_index = simulation_dates.get_loc(date)

  # Estimate r
  delta = len(product_lifetime[date:])
  tau = delta / 252
  curve_fit = curves[date_index]
  r_yearly = curve_fit(tau) / 100

  # Set sigma to the daily implied volatility rate
  sigma_yearly = get_implied_volatility_rate(date) / 100
  sigma_daily = sigma_yearly / (252**(1/2))

  # We need the daily rate for the simulations
  r_daily = r_yearly / 252

  # We need to sample another Z, independently, for simulating the initial pilot prices
  Z_pilot = np.random.normal(0, 1, size=(num_sim, num_periods))
  Z_new = Z_matrix[date_index]
  assert Z_pilot.shape == Z_new.shape

  # Simulate payoff and expected value
  payoffs = simulate_GBM_CV(num_sim, prices[-1], r_daily, sigma_daily, delta_t, tau, Z_pilot=Z_pilot, Z_new=Z_new)
  expected_value = np.mean(payoffs)
  
  GBM_Vol_CV_expected_values.append(expected_value)

#### 6.5.1 Plotting the Results (aka The Moment of Truth)

In [ ]:
simulation_reults = pd.DataFrame(index=simulation_dates)
product_prices = product_prices[product_prices.index >= start_date]
assert len(product_prices) == len(simulation_reults)

# Gather data into a single dataframe
simulation_reults['Actual'] = product_prices["value"]
simulation_reults['GBM_Div'] = GBM_Div_expected_values
simulation_reults['Implied_Vol'] = GBM_Vol_expected_values
simulation_reults['Implied_Vol_AV'] = GBM_Vol_AV_expected_values
simulation_reults['Implied_Vol_EMS'] = GBM_Vol_EMS_expected_values
simulation_reults['Implied_Vol_CV'] = GBM_Vol_CV_expected_values

simulation_reults.head(10)

In [ ]:
# Save results
simulation_reults.to_csv(os.path.join(SAVE_DIR, "Implied_Volatility_Prices.csv"))

In [ ]:
plt.figure(figsize=(14,8))
plt.title('Backtesting GBM and Implied Volatility', fontdict={"fontsize":18})

simulation_reults['Actual'].plot(legend=True)
simulation_reults['GBM_Div'].plot(legend=True, style="--")
simulation_reults['Implied_Vol'].plot(legend=True, style="--")

plt.xlabel('Market Days')
plt.ylabel('Derivative Price in USD')

In [ ]:
plt.figure(figsize=(14,8))
plt.title('Variance Reduction on the Implied Volatility Model', fontdict={"fontsize":18})

simulation_reults['Actual'].plot(legend=True)
simulation_reults['Implied_Vol'].plot(legend=True, style="--")
simulation_reults['Implied_Vol_AV'].plot(legend=True, style="--")
simulation_reults['Implied_Vol_EMS'].plot(legend=True, style="--")
simulation_reults['Implied_Vol_CV'].plot(legend=True, style="--")

plt.xlabel('Market Days')
plt.ylabel('Derivative Price in USD')

#### 6.5.2 Computing the Prediction Errors

In [ ]:
mean_abs_error=mean_absolute_error(simulation_reults['Actual'].to_list(),simulation_reults['GBM_Div'].to_list())
mean_sq_error=mean_squared_error(simulation_reults['Actual'].to_list(),simulation_reults['GBM_Div'].to_list())
print(f'Mean Absolute Error for GBM with div:\t\t{mean_abs_error:.4f}')
print(f'Mean Squared Error for GBM with div:\t\t{mean_sq_error:.4f}\n')

mean_abs_error=mean_absolute_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol'].to_list())
mean_sq_error=mean_squared_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol'].to_list())
print(f'Mean Absolute Error for Implied Vol:\t{mean_abs_error:.4f}')
print(f'Mean Squared Error for Implied Vol:\t{mean_sq_error:.4f}\n')

mean_abs_error=mean_absolute_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol_AV'].to_list())
mean_sq_error=mean_squared_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol_AV'].to_list())
print(f'Mean Absolute Error for Implied Vol with AV:\t{mean_abs_error:.4f}')
print(f'Mean Squared Error for Implied Vol with AV:\t{mean_sq_error:.4f}\n')

mean_abs_error=mean_absolute_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol_EMS'].to_list())
mean_sq_error=mean_squared_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol_EMS'].to_list())
print(f'Mean Absolute Error for Implied Vol with EMS:\t{mean_abs_error:.4f}')
print(f'Mean Squared Error for Implied Vol with EMS:\t{mean_sq_error:.4f}\n')   

mean_abs_error=mean_absolute_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol_CV'].to_list())
mean_sq_error=mean_squared_error(simulation_reults['Actual'].to_list(),simulation_reults['Implied_Vol_CV'].to_list())
print(f'Mean Absolute Error for Implied Vol with CV:\t{mean_abs_error:.4f}')
print(f'Mean Squared Error for Implied Vol with CV:\t{mean_sq_error:.4f}\n')

### 7. Estimating Sensitivities

The simulation results from the three scenarios; S, S+h, and S-h, are used to estimate delta and gamma for each time point by Finite-difference Method.

In [ ]:
@njit
def simulate_GBM_finite_difference(num_sim, S_0, r, sigma, delta_t, T, Z, h):
  # Initializations
  v = r - 0.5*(sigma**2)
  num_periods = int(T/delta_t)

  # Generate S+h and S-h matrices too
  S_matrix = np.full(shape=(num_sim, num_periods + 1), fill_value = S_0, dtype=np.float64)
  S_plus_h_matrix = np.full(shape=(num_sim, num_periods + 1), fill_value = S_0 + h, dtype=np.float64)
  S_minus_h_matrix = np.full(shape=(num_sim, num_periods + 1), fill_value = S_0 - h, dtype=np.float64)

  # We are using the form derived by Ito's lemma 
  for i in range(num_sim):
    for j in range(1, num_periods + 1):
      log_diff = v*delta_t + (sigma*np.sqrt(delta_t) * Z[i][j-1])
      S_matrix[i][j] = S_matrix[i][j-1]*np.exp(log_diff)
      S_plus_h_matrix[i][j] = S_plus_h_matrix[i][j-1]*np.exp(log_diff)
      S_minus_h_matrix[i][j] = S_minus_h_matrix[i][j-1]*np.exp(log_diff)

  return S_matrix, S_plus_h_matrix, S_minus_h_matrix

In [ ]:
# Simulating Delta with Finite-difference method
GBM_Vol_deltas = []
GBM_Vol_gammas = []

for date, prices in zip(simulation_dates, backtest_windows):
  # Convert date to index (so we can lookup prices and interest rate curves)
  date_index = simulation_dates.get_loc(date)

  # Estimate r
  delta = len(product_lifetime[date:])
  tau = delta / 252
  curve_fit = curves[date_index]
  r_yearly = curve_fit(tau) / 100

  # Set sigma to the daily implied volatility rate
  sigma_yearly = get_implied_volatility_rate(date) / 100
  sigma_daily = sigma_yearly / (252**(1/2))

  # We need the daily rate for the simulations
  r_daily = r_yearly / 252

  # CHECK: Set h (0.005 worked well)
  h = 0.005
  #h = prices[-1] * 0.01

  # Simulate prices and base payoffs
  simulated_prices, simulated_prices_plus_h, simulated_prices_minus_h = simulate_GBM_finite_difference(num_sim, prices[-1], r_daily, sigma_daily, delta_t, tau, Z=Z_matrix[date_index], h=h)
  payoffs = np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices)
  payoffs_plus_h = np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices_plus_h)
  payoffs_minus_h = np.apply_along_axis(func1d=calculate_payoff, axis=1, arr=simulated_prices_minus_h)

  # Caclulate and collect expected values
  expected_value = np.exp(-r_yearly*tau) * np.mean(payoffs)
  expected_value_plus_h = np.exp(-r_yearly*tau) * np.mean(payoffs_plus_h)
  expected_value_minus_h = np.exp(-r_yearly*tau) * np.mean(payoffs_minus_h)

  # Compute Delta and Gamma (using FDM)
  delta = (expected_value_plus_h - expected_value_minus_h)/(2*h)
  gamma = (expected_value_plus_h - 2*expected_value + expected_value_minus_h)/(h**2)

  GBM_Vol_deltas.append(delta)
  GBM_Vol_gammas.append(gamma)

In [ ]:
# Create a DataFrame for results
greeks_results = pd.DataFrame(index=simulation_dates)
greeks_results['Delta'] = GBM_Vol_deltas
greeks_results['Gamma'] = GBM_Vol_gammas

print(f"Average Delta: {np.mean(GBM_Vol_deltas):.4f}")
print(f"Average Gamma: {np.mean(GBM_Vol_gammas):.4f}")

In [ ]:
# Plot Delta
plt.figure(figsize=(8, 6))
plt.plot(greeks_results.index, greeks_results['Delta'], label='Delta', color='tab:orange', linestyle='--')
plt.title('Delta', fontsize=16)
plt.xlabel('Market Days', fontsize=14)
plt.ylabel('Delta Value in USD', fontsize=14)
plt.legend()
plt.show()

# Plot Gamma
plt.figure(figsize=(8, 6))
plt.plot(greeks_results.index, greeks_results['Gamma'], label='Gamma', color='tab:blue', linestyle='--')
plt.title('Gamma', fontsize=16)
plt.xlabel('Market Days', fontsize=14)
plt.ylabel('Gamma Value in USD', fontsize=14)
plt.legend()
plt.show()

In [ ]:
# Save results
greeks_results.to_csv(os.path.join(SAVE_DIR, "Implied_Volatility__Greeks_0.005.csv"))

That's about it for the Implied Volatility model. Feel free to explore the other notebooks in `/models` for more compliated models.

#### Thank you!